Частота агрегации (resampling):
При работе с временными рядами часто требуется изменить частоту данных — например, из минутных данных сделать ежедневные или месячные. Для этого в pandas существует метод resample(). Этот метод позволяет изменять частоту данных и применять к ним агрегирующие функции.

df_resampled = df.resample('D').mean()  # Среднее значение по дням

In [ ]:
df_resampled = df.resample('W').sum()  # Сумма за неделю

Агрегация с использованием временных окон:
Иногда требуется агрегация на основе окон (например, среднее за последние 7 дней). В таком случае полезен метод rolling(), который создает скользящее окно, по которому можно посчитать среднее, сумму или другие статистики.

In [ ]:
df['7_day_avg'] = df['sales'].rolling(window=7).mean()  # Скользящее среднее за 7 дней

Месячная агрегация с вычислением трендов и сезонности:
Для временных рядов важны такие аспекты, как тренды и сезонность. Агрегация по месяцам позволяет выделить сезонные колебания и тренды, например, если продажи растут в зимний или летний период.

In [ ]:
df['month'] = df['date'].dt.month
monthly_sales = df.groupby('month')['sales'].sum()  # Сумма продаж по месяцам
print(monthly_sales)

Индикаторы по времени:
Важно также извлекать дополнительные индикаторы времени из временных меток, такие как:

    День недели: Определяет, насколько продажи или другие события зависят от дня недели.

    Праздники и выходные: Сезонность может зависеть от праздников или выходных.

    Час дня: Для анализа дневных временных рядов.

In [ ]:
df['day_of_week'] = df['date'].dt.dayofweek  # День недели (0 - понедельник, 6 - воскресенье)

Использование агрегации для создания фич:
Агрегированные данные помогают создавать новые признаки для моделей:

    Среднее значение за прошлые периоды

    Разница между текущим и предыдущим значением

    Медицинские показатели для сезонных данных (например, периодические пики)

In [ ]:
df['sales_diff'] = df['sales'].diff()  # Разница между текущим и предыдущим значением

Частотная агрегация с пропусками данных:
В реальных временных рядах часто бывают пропуски. При агрегации важно правильно работать с ними:

    Пропуски можно заполнять с помощью метода fillna() перед агрегацией.

    Важно учитывать, как пропуски могут повлиять на вычисление статистик, например, при расчете среднего или суммы.

Применение скользящего окна:

    Январь: нет, так как требуется 3 месяца.

    Февраль: нет, так как требуется 3 месяца.

    Март: среднее по январю, февралю и марту = (100 + 120 + 150) / 3 = 123.33

    Апрель: среднее по февралю, марту и апрелю = (120 + 150 + 130) / 3 = 133.33

    Май: среднее по марту, апрелю и маю = (150 + 130 + 170) / 3 = 150

    И так далее...

Таким образом, каждый месяц после третьего (или после того, как скользящее окно имеет достаточную длину) будет вычисляться статистика для окна из предыдущих значений.
Применение в коде (например, с Pandas):

In [ ]:
import pandas as pd

# Пример данных
data = {'Месяц': ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'],
        'Продажа': [100, 120, 150, 130, 170, 160, 180, 200, 210, 190, 220, 250]}

df = pd.DataFrame(data)

# Применение rolling с окном 3 и вычисление среднего
df['Скользящее среднее'] = df['Продажа'].rolling(window=3).mean()

print(df)

LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Пример данных (продажа за 24 месяца)
data = {'Месяц': ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь', 
                  'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь'],
        'Продажа': [100, 120, 150, 130, 170, 160, 180, 200, 210, 190, 220, 250, 240, 260, 270, 280, 300, 310, 320, 330, 350, 360, 380, 390]}

df = pd.DataFrame(data)

# Используем только колонку 'Продажа' для прогноза
sales = df['Продажа'].values
sales = sales.reshape(-1, 1)  # Делаем массив 2D для нормализации

# Нормализуем данные
scaler = MinMaxScaler(feature_range=(0, 1))
sales_scaled = scaler.fit_transform(sales)

# Функция для создания данных с использованием rolling
def create_rolling_dataset(data, window_size=12):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i, 0])  # Входные данные для окна (предыдущие месяцы)
        y.append(data[i, 0])  # Целевое значение (следующий месяц)
    return np.array(X), np.array(y)

# Создаем обучающие данные с использованием скользящего окна
window_size = 12  # Размер окна - 12 месяцев для прогноза следующего месяца
X, y = create_rolling_dataset(sales_scaled, window_size)

# Переводим X в формат (samples, time_step, features) для LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Разделим на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [ ]:
# Создаем модель
model = Sequential()

# Добавляем LSTM слой
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))

# Добавляем слой Dense для предсказания
model.add(Dense(units=1))

# Компилируем модель
model.compile(optimizer='adam', loss='mean_squared_error')

# Обучаем модель
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), verbose=1)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Прогнозируем на тестовых данных
predictions = model.predict(X_test)

# Преобразуем прогнозы обратно в оригинальный масштаб
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# Оценка модели (например, MSE)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test_rescaled, predictions_rescaled)
print(f'Mean Squared Error: {mse}')

# Визуализация с Seaborn
plt.figure(figsize=(12,6))

# Реальные данные
sns.lineplot(x=np.arange(len(y_test_rescaled)), y=y_test_rescaled.flatten(), label='Real Sales', color='blue')

# Предсказанные данные
sns.lineplot(x=np.arange(len(predictions_rescaled)), y=predictions_rescaled.flatten(), label='Predicted Sales', color='red')

# Оформление графика
plt.title('Sales Prediction using LSTM')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()


Пример с четырьмя подграфиками:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))  # 2 строки, 2 столбца

# График 1: Реальные и предсказанные продажи
sns.lineplot(x=np.arange(len(y_test_rescaled)), y=y_test_rescaled.flatten(), label='Real Sales', color='blue', ax=axes[0, 0])
sns.lineplot(x=np.arange(len(predictions_rescaled)), y=predictions_rescaled.flatten(), label='Predicted Sales', color='red', ax=axes[0, 0])
axes[0, 0].set_title('Sales Prediction')

# График 2: Ошибки прогноза
errors = y_test_rescaled.flatten() - predictions_rescaled.flatten()
sns.lineplot(x=np.arange(len(errors)), y=errors, label='Prediction Errors', color='green', ax=axes[0, 1])
axes[0, 1].set_title('Prediction Errors')

# График 3: МАЕ (Mean Absolute Error) по времени
mae = np.abs(errors)
sns.lineplot(x=np.arange(len(mae)), y=mae, label='MAE', color='orange', ax=axes[1, 0])
axes[1, 0].set_title('Mean Absolute Error')

# График 4: МСЕ (Mean Squared Error) по времени
mse = errors**2
sns.lineplot(x=np.arange(len(mse)), y=mse, label='MSE', color='purple', ax=axes[1, 1])
axes[1, 1].set_title('Mean Squared Error')

plt.tight_layout()
plt.show()


Конечно! Давай создадим более сложный и гибкий **пайплайн для временных рядов**, который будет включать все основные этапы:

1. **Предобработка данных** (обработка пропусков, нормализация).
2. **Инженерия признаков** (например, добавление временных признаков).
3. **Подготовка данных для модели** (создание временных окон для LSTM или других моделей).
4. **Моделирование** (например, LSTM, ARIMA, Prophet или другие модели).
5. **Оценка модели** (метрики, визуализация).
6. **Прогнозирование и тестирование**.

Вот как это может выглядеть для временных рядов с использованием **LSTM** как модели:

### Шаг 1: Импорты и подготовка

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from statsmodels.tsa.stattools import adfuller  # Для проверки на стационарность
```

### Шаг 2: Предобработка данных

1. **Загрузка данных**
2. **Обработка пропусков**
3. **Проверка на стационарность**
4. **Нормализация данных**

```python
# Загружаем данные
def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
    return df

# Обработка пропусков
def handle_missing_data(df):
    df.fillna(method='ffill', inplace=True)  # Заполнение пропусков методом forward fill
    return df

# Проверка на стационарность
def check_stationarity(df):
    result = adfuller(df)
    print(f'p-value: {result[1]}')
    if result[1] > 0.05:
        print("Временной ряд нестационарен, нужно применять преобразования.")
    else:
        print("Ряд стационарен.")

# Нормализация данных
def normalize_data(df):
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = scaler.fit_transform(df.values.reshape(-1, 1))  # Нормализация в диапазон [0, 1]
    return df_scaled, scaler

# Загрузка данных
df = load_data('sales_data.csv')  # Пример: путь к твоим данным

# Обработка пропусков
df = handle_missing_data(df)

# Проверка на стационарность
check_stationarity(df['Sales'])

# Нормализация данных
sales_scaled, scaler = normalize_data(df['Sales'])
```

### Шаг 3: Инженерия признаков

Добавим дополнительные признаки, которые могут быть полезны для модели. Например:

* **Временные признаки** (например, месяц, день недели).
* **Скользящие средние и другие агрегаты**.

```python
# Добавление временных признаков
def add_time_features(df):
    df['Month'] = df.index.month
    df['Quarter'] = df.index.quarter
    df['Year'] = df.index.year
    df['DayOfWeek'] = df.index.dayofweek
    return df

df = add_time_features(df)

# Создание скользящих средних и других агрегатов
def add_rolling_features(df, window_size=12):
    df['RollingMean'] = df['Sales'].rolling(window=window_size).mean()
    df['RollingStd'] = df['Sales'].rolling(window=window_size).std()
    return df

df = add_rolling_features(df)
```

### Шаг 4: Подготовка данных для модели LSTM

1. **Создание окон** (например, 12 месяцев для прогноза следующего месяца).
2. **Разделение на тренировочные и тестовые данные**.

```python
# Функция для создания последовательностей для LSTM
def create_lstm_dataset(data, time_step=12):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])  # Окно из 12 месяцев
        y.append(data[i, 0])  # Следующее значение
    return np.array(X), np.array(y)

# Создание данных
time_step = 12  # Используем 12 месяцев для предсказания следующего
X, y = create_lstm_dataset(sales_scaled, time_step)

# Переформатируем X для LSTM: (samples, time_steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Разделяем данные на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
```

### Шаг 5: Моделирование (LSTM)

Создадим и обучим модель LSTM.

```python
# Создание модели LSTM
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Создание и обучение модели
model = create_lstm_model()
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), verbose=1)
```

### Шаг 6: Оценка модели и прогнозирование

```python
# Прогнозируем на тестовых данных
predictions = model.predict(X_test)

# Преобразуем прогнозы обратно в оригинальный масштаб
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# Оценка модели (MSE)
mse = mean_squared_error(y_test_rescaled, predictions_rescaled)
print(f'Mean Squared Error: {mse}')

# Визуализация результатов
plt.figure(figsize=(12, 6))
sns.lineplot(x=np.arange(len(y_test_rescaled)), y=y_test_rescaled.flatten(), label='Real Sales', color='blue')
sns.lineplot(x=np.arange(len(predictions_rescaled)), y=predictions_rescaled.flatten(), label='Predicted Sales', color='red')
plt.title('Sales Prediction using LSTM')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.legend()
plt.show()
```

### Шаг 7: Прогнозирование на будущем

```python
# Прогнозирование на будущее (например, на 6 месяцев вперед)
def predict_future(model, last_data, future_steps=6):
    future_predictions = []
    current_data = last_data
    for _ in range(future_steps):
        pred = model.predict(current_data.reshape(1, current_data.shape[0], 1))
        future_predictions.append(pred)
        current_data = np.append(current_data[1:], pred, axis=0)
    return np.array(future_predictions)

# Прогнозирование на 6 месяцев вперед
future_steps = 6
last_data = sales_scaled[-time_step:]
future_predictions = predict_future(model, last_data, future_steps)

# Преобразуем прогнозы обратно в оригинальный масштаб
future_predictions_rescaled = scaler.inverse_transform(future_predictions)

# Визуализация прогноза
plt.figure(figsize=(12, 6))
plt.plot(np.arange(len(sales_rescaled)), sales_rescaled, label='Historical Sales')
plt.plot(np.arange(len(sales_rescaled), len(sales_rescaled) + future_steps), future_predictions_rescaled.flatten(), label='Future Predictions', color='orange')
plt.title('Future Sales Prediction')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.legend()
plt.show()

df_future_predictions = pd.DataFrame(future_predictions_rescaled, columns=["Predicted_Sales"])
df_future_predictions.to_csv('future_sales_predictions.csv', index=False)
```

---

### Полный пайплайн:

1. **Загрузка и предобработка данных**.
2. **Инженерия признаков**: добавление временных признаков и скользящих агрегатов.
3. **Создание и обучение модели LSTM**.
4. **Оценка модели**: метрики, визуализация.
5. **Прогнозирование будущих значений**.

Это довольно полный пайплайн, который можно адаптировать под любые задачи прогнозирования временных рядов. Если требуется добавить другие модели или улучшить отдельные части пайплайна (например, через кросс-валидацию, тюнинг гиперпараметров), дайте знать!
